# Setup

**Note to Mac users:**
* You might get this error: `Unable to revert mtime: /Library/Fonts`
* The solution is to install the *libmagic* brew: `brew install libmagic`
* Reference: [Python MacOS Error: Unable to revert mtime: /Library/Fonts](https://stackoverflow.com/questions/62279920/python-macos-error-unable-to-revert-mtime-library-fonts)

In [ ]:
from setup import *

## Set Variables

### Instructions

Gather the following variables:

* database name
* path to the Net-Manage repository
* private data directory
* path to store the output
* Meraki API key (if applicable)
* Credentials to login to devices (if applicable)

**Note:** The inventory file is statically set to *private_data_dir/inventory/hosts*

#### Explanation of Variables

### database name

defaults to the current date, in YYYY-MM-DD format--e.g., ```2023-01-05.db```

##### path to the Net-Manage repository

defaults to ```~/source/repos/InsightSSG/Net-Manage```

##### Path to the Ansible Private Data Directory

This is the folder that contains the inventory folder (not the host file).<br><br>
For example, If your folder structure looks like this:```tenants/client_a/inventory/hosts```<br><br>
Then the private data directory would be```tenants/client_a```

##### path to store the output

Path to store the database and other script output. Defaults to 'private_data_dir/output'

**note:** Ansible-runner output, like artifacts, will still be stored in the private data directory

##### Example folder structure

* Net-Manage path: *~/source/repos/InsightSSG/Net-Manage*

* Private data directory: *~/tenant_data/*

* Inventory file: *~/tenant_data/inventory/hosts*

### Enter Variables

In [ ]:
from getpass import getpass
api_key, db, db_path, inventories, npm_server, npm_username, npm_password, nm_path, out_path, private_data_dir = hp.set_vars()
play_path = f'{nm_path}/playbooks'
username, password = hp.get_creds('device')
infoblox_host = input('Enter the IP or FQDN of the Infoblox host: ')
infoblox_username, infoblox_password = hp.get_creds('Infoblox')
nb_path = input('Enter the IP or FQDN of the Netbox host: ')
nb_token = getpass('Enter your Netbox API token: ')

# Select and Run Collectors

## Select the hostgroups to run the collectors on

**Note:**

* If running this in a miniconda environment and the widget does not display, run the command below.
* See: https://stackoverflow.com/questions/36351109/ipython-notebook-ipywidgets-does-not-show

```
jupyter nbextension enable --py --sys-prefix widgetsnbextension
```

In [ ]:
hostgroup_select = select_hostgroups(private_data_dir)
for key in sorted(hostgroup_select.keys()):
    print(key.upper())
    display(widgets.GridBox(hostgroup_select[key],
                            layout=widgets.Layout(grid_template_columns='repeat(3, 200px)')))
    print('\n')

## Select the collectors to run

In [ ]:
collector_select = select_collectors(collector_select, hostgroup_select)
for key in sorted(collector_select.keys()):
    print(key.upper())
    display(widgets.GridBox(collector_select[key],
                            layout=widgets.Layout(grid_template_columns='repeat(3, 250px)')))
    print('\n')

### Select Meraki Organizations and Networks (if applicable)

#### Debugging

In [ ]:
# This cell is only here for debugging

import run_collectors as rc
from collectors import collectors as cl
from collectors import meraki_collectors as mc
from helpers import helpers as hp

importlib.reload(cl)
importlib.reload(hp)
importlib.reload(mc)
importlib.reload(rc)

#### Instructions

**Organizations**

A comma-delimited list of organizations. If left blank, all organizations will be queried.

**Networks**

A comma-delimited list of network IDs. Defaults to 'all', but some collectors require one or more network IDs.

**MAC Addresses**

A comma-delimited list of MAC addresses. Used for filtering the output of some collectors, like ```meraki_get_network_clients```

**Lookback Timespan**

The lookback timespan in seconds. Defaults to 86400 (1 day). Formulas are accepted, like ```60*60*24*30```

**Number of Results Per Page**

The number of results per page to return. Accepted values are 3-1000. Defaults to ```1000```.

**Total Number of Pages to Return**

The total number of pages to return. Defaults to ```all```

#### Enter Variables

In [ ]:
for collector in collector_select['meraki']:
    if collector.value:
        orgs, networks, macs, timespan, per_page, total_pages = hp.get_user_meraki_input()
        break

## Run Collectors

In [ ]:
%%time

# Reload imported modules (used for debugging)
import run_collectors as rc
from collectors import collectors as cl
from collectors import infoblox_nios_collectors as nc
from collectors import meraki_collectors as mc
from collectors import netbox_collectors as nbc
from collectors import solarwinds_collectors as swc
from helpers import helpers as hp
importlib.reload(cl)
importlib.reload(hp)
importlib.reload(rc)
importlib.reload(nc)
importlib.reload(mc)
importlib.reload(nbc)
importlib.reload(swc)

# Create a dataframe of collectors to run
print('COLLECTORS TO RUN\n')
df_collectors = create_collectors_df(collector_select, hostgroup_select)
display(df_collectors.style.hide(axis="index"))

# Set the timestamp so it will be consistent for all collectors
ts = dt.datetime.now()
ts = ts.strftime('%Y-%m-%d_%H%M')

# Execute the collectors
for idx, row in df_collectors.iterrows():
    ansible_os = row['ansible_os']
    hostgroup = row['hostgroup']
    collector = row['collector']
    result = rc.collect(collector,
                        nm_path,
                        private_data_dir,
                        ts,
                        ansible_os=ansible_os,
                        username=username,
                        password=password,
                        api_key=api_key,
                        hostgroup=hostgroup,
                        play_path=play_path,
                        db_path=db_path,
                        infoblox_host=infoblox_host,
                        infoblox_user=infoblox_username,
                        infoblox_pass=infoblox_password,
                        infoblox_paging=True,
                        nb_path=nb_path,
                        nb_token=nb_token,
                        npm_password=npm_password,
                        npm_server=npm_server,
                        npm_username=npm_username,
                        orgs=orgs,
                        networks=networks,
                        macs=macs,
                        timespan=timespan,
                        per_page=per_page,
                        total_pages=total_pages,
                        validate_certs=False,
                        method='append')
    print(f'\nRESULT: {ansible_os.upper()} {collector.upper()} COLLECTOR\n')
    display(result)

# Run Validators

## Reload imported modules (optional; used for testing)

In [ ]:
import run_collectors as rc
import validators as vl
from collectors import collectors as cl
from collectors import meraki_collectors as mc
from helpers import helpers as hp

importlib.reload(cl)
importlib.reload(mc)
importlib.reload(hp)
importlib.reload(rc)
importlib.reload(vl)

## F5 Validators

### ```f5_vip_availability```

In [ ]:
df_diff = vl.f5_vip_availability(db_path, 'f5_vip_availability')
display(df_diff)

### ```f5_pool_availability```

In [ ]:
df_diff = vl.f5_pool_availability(db_path, 'f5_pool_availability')
display(df_diff)

### ```f5_pool_member_availability```

In [ ]:
df_diff = vl.f5_pool_member_availability(db_path, 'f5_pool_member_availability')
display(df_diff)

## Meraki Validators

### ```meraki_get_org_device_statuses```

In [ ]:
%%time
df_diff = vl.meraki_device_statuses_availability(db_path, 'meraki_get_org_device_statuses')
if len(df_diff) > 0:
    display(df_diff.sort_values(by='networkId', ascending=True).style.hide(axis="index"))
else:
    display(df_diff)